### Use Case:

Dataset: 
The dataset is provided by Sebastian Tomczak and collected from Emerging Markets Information Service (EMIS) : https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data


The dataset is about bankruptcy prediction of companies. In theses datasets, we retrieve information about emerging markets around the world. The dataset is composed of thousands of rows where each row corresponds to a company. The attribute about theses companies is given in data_description.txt file:

Here we will use teh 1st and 2nd year data to train a classifier model and the 3rd year data to predict 

The goal of this project is to explore and analyse the capabities of Sagemaker Feature Store: Mainly what are the advantage of using it for an ML usecase

- In total in this dataset, we have more than 43,000 companies' status inequitably distributed on 5 years. 
- The columns represent the 64 variables we will use to predict bankruptcy. 
- Among these variables, we can see there is underlying variables which appear and impact a lot of variables, like total assets, total liabilities, net profit and much more that we can use directly in order to have fewer variables to include in our model without loss of information. 
- We can also see there is multiple missing values symbolised by a "?" in many fields.



#### Required imports

In [2]:
import sagemaker
import sys

import boto3
import pandas as pd
import numpy as np
import io
import time
from time import gmtime, strftime, sleep
import datetime

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

Importing and organizing the data
Dataset Link:The Dataset can be found at Polish bankruptcy dataset

There are a total of 5 .arff data files with the names 1year, 2year, 3year, 4year, 5year.
Load the .arff files and convert them into pandas dataframes using the load_dataframes function.
Give them new columns headers using the function set_new_headers. The column labels for the features are like X1, X2, ... , X64. The class label is Y.
Print the first 5 rows of a dataframe, to see how the data looks like.

In [3]:
############################################################
year = "1year"
# Loads the 5 raw .arff files into pandas dataframes
def load_dataframes():
    data_loc = f's3://sagemaker-featurestore-pov/bankruptcy data/{year}.csv'
    return pd.read_csv(data_loc)

# dataframes is the list of pandas dataframes for the 5 year datafiles.  
dataframe = load_dataframes()


In [4]:
dataframe

,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65
0,0.200550,0.37951,0.39641,2.04720,32.3510,0.388250,0.249760,1.330500,1.13890,0.504940,...,0.121960,0.397180,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.42770,0
1,0.209120,0.49988,0.47225,1.94470,14.7860,0.000000,0.258340,0.996010,1.69960,0.497880,...,0.121300,0.420020,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.98700,0
2,0.248660,0.69592,0.26713,1.55480,-1.1523,0.000000,0.309060,0.436950,1.30900,0.304080,...,0.241140,0.817740,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.20780,0
3,0.081483,0.30734,0.45879,2.49280,51.9520,0.149880,0.092704,1.866100,1.05710,0.573530,...,0.054015,0.142070,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.54970,0
4,0.187320,0.61323,0.22960,1.40630,-7.3128,0.187320,0.187320,0.630700,1.15590,0.386770,...,0.134850,0.484310,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.89800,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7022,0.018371,0.47410,-0.13619,0.60839,-18.4490,0.018371,0.018371,0.972030,1.01210,0.460840,...,0.011909,0.039866,0.98809,0.274140,73.5050,79.2370,31.268,11.6730,5.14890,1
7023,-0.013359,0.58354,-0.02265,0.92896,-42.2320,-0.013359,-0.015036,0.562890,0.98904,0.328470,...,-0.011082,-0.040671,1.01110,0.805920,10.5990,7.1740,94.092,3.8792,1.75720,1
7024,0.006338,0.50276,0.43923,1.87360,9.7417,0.006338,0.012022,0.983560,1.00830,0.494490,...,0.008258,0.012817,0.99174,0.000000,10.4700,6.0759,51.019,7.1542,62.00100,1
7025,-0.041643,0.84810,-0.12852,0.57485,-121.9200,0.000000,-0.036795,0.179010,0.42138,0.151820,...,-0.232720,-0.274290,0.98788,3.593100,39.7030,3.1420,261.850,1.3939,0.51005,1


In [5]:
file_loc = 's3://sagemaker-featurestore-pov/bankruptcy data/data_description.txt'
description = pd.read_csv(file_loc)
description

,Attribute Information:
0,X1 net profit / total assets
1,X2 total liabilities / total assets
2,X3 working capital / total assets
3,X4 current assets / short-term liabilities
4,X5 [(cash + short-term securities + receivable...
...,...
59,X60 sales / inventory
60,X61 sales / receivables
61,X62 (short-term liabilities *365) / sales
62,X63 sales / short-term liabilities


In [6]:
dataframe.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
1,7024.0,0.034660,4.565504,-2.568900e+02,0.021182,0.075802,0.160268,94.28
2,7024.0,0.560215,5.350084,-7.216200e+01,0.296678,0.482960,0.680233,441.50
3,7024.0,0.119969,5.275459,-4.405000e+02,0.026968,0.181275,0.362548,1.00
4,6997.0,2.629143,13.257356,0.000000e+00,1.063100,1.502000,2.460700,1017.80
5,7019.0,-263.167247,37074.596449,-2.722100e+06,-44.498000,-5.373900,37.770500,990900.00
...,...,...,...,...,...,...,...,...
61,7005.0,15.642228,261.554534,1.600000e-05,4.829000,7.033700,10.703000,21110.00
62,7027.0,4763.201955,310783.483587,0.000000e+00,43.222500,68.509000,106.335000,25016000.00
63,6997.0,8.126852,19.996419,1.500000e-05,3.425400,5.303200,8.357900,1042.20
64,6993.0,208.731950,5140.708804,1.000000e-05,2.538600,4.637700,9.782200,294770.00


In [7]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7027 entries, 0 to 7026
Data columns (total 65 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       7024 non-null   float64
 1   2       7024 non-null   float64
 2   3       7024 non-null   float64
 3   4       6997 non-null   float64
 4   5       7019 non-null   float64
 5   6       7024 non-null   float64
 6   7       7024 non-null   float64
 7   8       7002 non-null   float64
 8   9       7026 non-null   float64
 9   10      7024 non-null   float64
 10  11      6988 non-null   float64
 11  12      6997 non-null   float64
 12  13      7027 non-null   float64
 13  14      7024 non-null   float64
 14  15      7025 non-null   float64
 15  16      7002 non-null   float64
 16  17      7002 non-null   float64
 17  18      7024 non-null   float64
 18  19      7027 non-null   float64
 19  20      7027 non-null   float64
 20  21      5405 non-null   float64
 21  22      7024 non-null   float64
 22  

In [8]:
dataframe = dataframe.fillna(dataframe.mean())

In [9]:
dataframe.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
1,7027.0,0.034660,4.564529,-2.568900e+02,0.021208,0.07579,0.16024,94.28
2,7027.0,0.560215,5.348941,-7.216200e+01,0.296760,0.48309,0.68017,441.50
3,7027.0,0.119969,5.274332,-4.405000e+02,0.027011,0.18118,0.36250,1.00
4,7027.0,2.629143,13.229022,0.000000e+00,1.064100,1.50520,2.47605,1017.80
5,7027.0,-263.167247,37053.483352,-2.722100e+06,-44.628500,-5.57530,37.69500,990900.00
...,...,...,...,...,...,...,...,...
61,7027.0,15.642228,261.144719,1.600000e-05,4.834250,7.04630,10.77550,21110.00
62,7027.0,4763.201955,310783.483587,0.000000e+00,43.222500,68.50900,106.33500,25016000.00
63,7027.0,8.126852,19.953682,1.500000e-05,3.432500,5.32780,8.32680,1042.20
64,7027.0,208.731950,5128.255341,1.000000e-05,2.550300,4.65740,9.91275,294770.00


In [10]:
dataframe['65'].value_counts()

0    6756
1     271
Name: 65, dtype: int64

In [11]:
# Adding columns with Id and Year for Record Identifier and Event Time
# Add event_time timestamp. This is a point in time when a new event occurs that corresponds to the creation or update of a record in a feature group
### timestamp = str(datetime.datetime.now())

dataframe['id'] = np.arange(len(dataframe))

current_time_sec = int(round(time.time()))
dataframe['year'] = year
dataframe['event_time'] = current_time_sec
dataframe = dataframe.astype({'event_time': 'float64'})
dataframe.rename({'65': 'labels'}, axis=1, inplace=True)

new_columns = ['id', 'event_time', 'year', 'labels']

In [14]:
def new_feature_addition(data):
            feature1 = data['6'] / data['7']
            feature2 = data['55'] / data['28']
            feature3 = (data['55'] * data['1']) / data['3']
            
            year = data['year']
            Id = data['id']
            EventTime = data['event_time']

            new_df = pd.DataFrame([Id, EventTime, year, feature1, feature2, feature3])
            new_df = new_df.T
            new_df.columns = ['id', 'event_time', 'year', 'attr66', 'attr67', 'attr68']

            new_df.replace([np.inf, -np.inf], np.nan, inplace=True)
            new_df = new_df.fillna(dataframe.mean())
            new_df = new_df.astype({'id':'int'})

            return (new_df)


In [15]:
df_new = new_feature_addition(dataframe)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


In [16]:
schema = ['7', '12', '15', '27', '41', '55']

for i in range(len(schema)):
    dataframe.rename({schema[i]: 'attr' + schema[i]}, axis=1, inplace=True)
    schema[i] = 'attr' + schema[i]
    i += i
    

schema.extend(new_columns)

In [17]:
#df = df.astype({'time': 'string'})
#df_new = df_new.astype({'time': 'string'})

In [18]:
df = dataframe[schema]

In [19]:
df

,attr7,attr12,attr15,attr27,attr41,attr55,id,event_time,year,labels
0,0.249760,0.659800,497.42,1.458200,0.051402,348690.0,0,1.676984e+09,1year,0
1,0.258340,0.516800,677.96,88.444000,0.064371,2304.6,1,1.676984e+09,1year,0
2,0.309060,0.641840,794.16,86.011000,0.074020,6332.7,2,1.676984e+09,1year,0
3,0.092704,0.301630,917.01,0.940760,0.069622,20545.0,3,1.676984e+09,1year,0
4,0.187320,0.331470,1133.20,1.413800,0.096680,3186.6,4,1.676984e+09,1year,0
...,...,...,...,...,...,...,...,...,...,...
7022,0.018371,0.052828,2041.50,1321.989035,0.158360,-2732.8,7022,1.676984e+09,1year,1
7023,-0.015036,-0.047162,4643.00,1321.989035,0.367780,-2557.1,7023,1.676984e+09,1year,1
7024,0.012022,0.023912,3465.60,1321.989035,0.297880,4826.9,7024,1.676984e+09,1year,1
7025,-0.036795,-0.121720,-23963.00,0.148430,0.933300,-1633.0,7025,1.676984e+09,1year,1


In [20]:
df_new

,id,event_time,year,attr66,attr67,attr68
0,0,1.676984e+09,1year,1.554492,197950.610275,176407.707929
1,1,1.676984e+09,1year,0.000000,135.996695,1020.514456
2,2,1.676984e+09,1year,0.000000,5959.066529,5894.842144
3,3,1.676984e+09,1year,1.616759,10472.525232,3648.876904
4,4,1.676984e+09,1year,1.000000,2849.248927,2599.799268
...,...,...,...,...,...,...
7022,7022,1.676984e+09,1year,1.000000,15821.223875,368.634032
7023,7023,1.676984e+09,1year,0.888468,79459.929772,-1508.180967
7024,7024,1.676984e+09,1year,0.527200,637.542761,69.651190
7025,7025,1.676984e+09,1year,-0.000000,10496.882432,-529.124020


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7027 entries, 0 to 7026
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attr7       7027 non-null   float64
 1   attr12      7027 non-null   float64
 2   attr15      7027 non-null   float64
 3   attr27      7027 non-null   float64
 4   attr41      7027 non-null   float64
 5   attr55      7027 non-null   float64
 6   id          7027 non-null   int64  
 7   event_time  7027 non-null   float64
 8   year        7027 non-null   object 
 9   labels      7027 non-null   int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 549.1+ KB


## Creating a Feature Group

In [22]:
timestamp=strftime('%Y-%m-%d-%H-%M-%S', gmtime())

feature_group_name = 'bankruptcy-fg-%s' % timestamp

In [23]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group1 = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

In [24]:
feature_group1.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='attr7', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr12', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr15', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr27', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr41', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr55', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='year', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='labels', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>)]

In [25]:
record_identifier_feature_name = 'id'
event_time_feature_name = 'event_time'

In [26]:
description='This feature group stores information of the important attributes that will be used for bankruptcy prediction.'
len(description)
# description has to be less than 128 characters

110

In [27]:
feature_group1.create(
    s3_uri=f's3://sagemaker-featurestore-pov/feature-group-storage',
    enable_online_store=True,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    description=description,
    role_arn=role)

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:820379933028:feature-group/bankruptcy-fg-2023-02-21-12-55-30',
 'ResponseMetadata': {'RequestId': 'f6b8b48a-a66d-42c5-b687-7c221dc5e71e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f6b8b48a-a66d-42c5-b687-7c221dc5e71e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Tue, 21 Feb 2023 12:55:49 GMT'},
  'RetryAttempts': 0}}

In [28]:
def check_feature_group_status(feature_group1):
    status = feature_group1.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group1.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group1.name} status - {status}.")

check_feature_group_status(feature_group1)

Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
FeatureGroup bankruptcy-fg-2023-02-21-12-55-30 status - Created.


### Creating the 2nd Feature Group with newly engineered features

In [29]:
timestamp=strftime('%Y-%m-%d-%H-%M-%S', gmtime())

feature_group_name = 'bankruptcy-new-features-%s' % timestamp

In [30]:
feature_group2 = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

In [31]:
feature_group2.load_feature_definitions(data_frame=df_new)

[FeatureDefinition(feature_name='id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='year', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='attr66', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr67', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='attr68', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>)]

In [32]:
record_identifier_feature_name = 'id'
event_time_feature_name = 'event_time'

In [33]:
description='This feature group stores information of the newly engineered attributes that will be used for bankruptcy prediction.'
len(description)
# description has to be less than 128 characters

117

In [34]:
feature_group2.create(
    s3_uri=f's3://sagemaker-featurestore-pov/feature-group-storage',
    enable_online_store=True,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    description=description,
    role_arn=role)

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:820379933028:feature-group/bankruptcy-new-features-2023-02-21-12-56-19',
 'ResponseMetadata': {'RequestId': '915f7f03-b446-46d8-bd82-9280aee9c3d9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '915f7f03-b446-46d8-bd82-9280aee9c3d9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Tue, 21 Feb 2023 12:56:44 GMT'},
  'RetryAttempts': 0}}

In [35]:
def check_feature_group_status(feature_group2):
    status = feature_group2.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group2.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group2.name} status - {status}.")

check_feature_group_status(feature_group2)

Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
FeatureGroup bankruptcy-new-features-2023-02-21-12-56-19 status - Created.


## Ingest data into a feature group: Batch Injest

In [36]:
feature_group1.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:820379933028:feature-group/bankruptcy-fg-2023-02-21-12-55-30',
 'FeatureGroupName': 'bankruptcy-fg-2023-02-21-12-55-30',
 'RecordIdentifierFeatureName': 'id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'attr7', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr12', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr15', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr27', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr41', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr55', 'FeatureType': 'Fractional'},
  {'FeatureName': 'id', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'Fractional'},
  {'FeatureName': 'year', 'FeatureType': 'String'},
  {'FeatureName': 'labels', 'FeatureType': 'Integral'}],
 'CreationTime': datetime.datetime(2023, 2, 21, 12, 55, 49, 372000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig

In [37]:
feature_group1.ingest(data_frame=df, max_workers=2, wait=True)

IngestionManagerPandas(feature_group_name='bankruptcy-fg-2023-02-21-12-55-30', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fa3b4163f90>, sagemaker_session=<sagemaker.session.Session object at 0x7fa3ec150550>, max_workers=2, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fa3b3e51590>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [38]:
feature_group2.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-central-1:820379933028:feature-group/bankruptcy-new-features-2023-02-21-12-56-19',
 'FeatureGroupName': 'bankruptcy-new-features-2023-02-21-12-56-19',
 'RecordIdentifierFeatureName': 'id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'id', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'Fractional'},
  {'FeatureName': 'year', 'FeatureType': 'String'},
  {'FeatureName': 'attr66', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr67', 'FeatureType': 'Fractional'},
  {'FeatureName': 'attr68', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2023, 2, 21, 12, 56, 44, 526000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-featurestore-pov/feature-group-storage',
   'ResolvedOutputS3Uri': 's3://sagemaker-featurestore-pov/feature-group-storage/820379933028/sagemaker/eu-central-1/of

In [39]:
feature_group2.ingest(data_frame=df_new, max_workers=2, wait=True)

IngestionManagerPandas(feature_group_name='bankruptcy-new-features-2023-02-21-12-56-19', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fa3b4163f90>, sagemaker_session=<sagemaker.session.Session object at 0x7fa3ec150550>, max_workers=2, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fa3b3aa2810>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

## Check the two feature groups

In [40]:
query = feature_group1.athena_query()
table_name = query.table_name
print(table_name)
query_string = ('SELECT * FROM "%s"' % table_name)
print('Running ' + query_string)

query.run(query_string=query_string,
          output_location=f's3://sagemaker-featurestore-pov/feature-group-storage')
query.wait()
dataset1 = query.as_dataframe()

bankruptcy-fg-2023-02-21-12-55-30-1676984149
Running SELECT * FROM "bankruptcy-fg-2023-02-21-12-55-30-1676984149"


In [41]:
dataset1

,attr7,attr12,attr15,attr27,attr41,attr55,id,event_time,year,labels,write_time,api_invocation_time,is_deleted
0,0.239510,0.582210,591.63,0.817450,0.055369,160590.00,3517,1.676984e+09,1year,0,2023-02-21 13:02:22.423,2023-02-21 12:57:14.000,False
1,0.005846,0.015571,10715.00,0.483730,0.807540,-77114.00,3546,1.676984e+09,1year,0,2023-02-21 13:02:22.423,2023-02-21 12:57:15.000,False
2,0.140900,0.265620,1839.40,0.494540,0.191350,8185.60,31,1.676984e+09,1year,0,2023-02-21 13:02:22.423,2023-02-21 12:57:15.000,False
3,0.106680,0.391730,902.39,1.226300,0.073469,14161.00,80,1.676984e+09,1year,0,2023-02-21 13:02:22.423,2023-02-21 12:57:15.000,False
4,-0.110380,-0.306440,-4326.20,-0.397840,-0.333680,2199.20,3650,1.676984e+09,1year,0,2023-02-21 13:02:22.423,2023-02-21 12:57:16.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7022,0.135900,0.276230,1516.60,17.876000,0.134360,1212.80,3121,1.676984e+09,1year,0,2023-02-21 13:02:22.451,2023-02-21 12:57:53.000,False
7023,0.043024,0.077069,2910.20,3.983600,0.247020,328.00,6818,1.676984e+09,1year,1,2023-02-21 13:02:22.451,2023-02-21 12:57:54.000,False
7024,0.085709,0.141330,3107.70,1.461400,0.175320,7306.40,3246,1.676984e+09,1year,0,2023-02-21 13:02:22.451,2023-02-21 12:57:55.000,False
7025,-0.014404,-0.022886,7554.80,1321.989035,0.216990,-6440.00,6926,1.676984e+09,1year,1,2023-02-21 13:02:22.451,2023-02-21 12:57:55.000,False


In [42]:
query = feature_group2.athena_query()
table_name = query.table_name
print(table_name)
query_string = ('SELECT * FROM "%s"' % table_name)
print('Running ' + query_string)

query.run(query_string=query_string,
          output_location=f's3://sagemaker-featurestore-pov/feature-group-storage')
query.wait()
dataset2 = query.as_dataframe()

bankruptcy-new-features-2023-02-21-12-56-19-1676984204
Running SELECT * FROM "bankruptcy-new-features-2023-02-21-12-56-19-1676984204"


In [43]:
dataset2

,id,event_time,year,attr66,attr67,attr68,write_time,api_invocation_time,is_deleted
0,4,1.676984e+09,1year,1.000000,2849.248927,2599.799268,2023-02-21 13:03:05.772,2023-02-21 12:58:04.000,False
1,3520,1.676984e+09,1year,2.486393,24973.864639,1101.680911,2023-02-21 13:03:05.772,2023-02-21 12:58:04.000,False
2,119,1.676984e+09,1year,0.013482,56851.883128,816.114200,2023-02-21 13:03:05.772,2023-02-21 12:58:06.000,False
3,3632,1.676984e+09,1year,0.000000,248.544469,2329.493853,2023-02-21 13:03:05.772,2023-02-21 12:58:06.000,False
4,3686,1.676984e+09,1year,0.317360,982.638567,846.597134,2023-02-21 13:03:05.772,2023-02-21 12:58:06.000,False
...,...,...,...,...,...,...,...,...,...
7022,3401,1.676984e+09,1year,0.000000,16954.867760,27.928344,2023-02-21 13:03:03.932,2023-02-21 12:58:47.000,False
7023,6961,1.676984e+09,1year,0.874589,30025.269455,143.557972,2023-02-21 13:03:03.932,2023-02-21 12:58:47.000,False
7024,6986,1.676984e+09,1year,0.003795,258.061128,628.736732,2023-02-21 13:03:03.932,2023-02-21 12:58:47.000,False
7025,3500,1.676984e+09,1year,9.433783,8265.346931,335.273649,2023-02-21 13:03:03.932,2023-02-21 12:58:48.000,False
